In [1]:
%run project.ipynb  # this loads the Project and ScanTailor classes
# right now I'm loading the local project in the project.ipynb, but
# I shouldn't do that there . . . I should do that here or by 
# parameterizing the notebooks with papermill!

Attributes of "Project" class . . . 
------------------------------------
project: test
identifier: 0012

Attributes of "ScanTailor" class . . . 
------------------------------------
project: agrutesc
identifier: 0012_004268
root_dir_path.is_dir(): /Users/dlisla/shared_with_docker
project_dir_path.is_dir(): /Users/dlisla/shared_with_docker/agrutesc
step0_dir_path.is_dir(): /Users/dlisla/shared_with_docker/agrutesc/0.ScanTailorFiles
step1_dir_path.is_dir(): /Users/dlisla/shared_with_docker/agrutesc/1.toScanTailor
step2_dir_path.is_dir(): /Users/dlisla/shared_with_docker/agrutesc/2.ScanTailored-holding
step3_dir_path.is_dir(): /Users/dlisla/shared_with_docker/agrutesc/3.toQC



In [2]:
from os.path import getmtime
from PIL import Image

In [3]:
# ============ functions ============ #
def copy_scantailor_images(from_dir_path, to_dir_path, decompress=True):
    
    # make sure we have an "out" directory
    scantailor_out_dir_path = from_dir_path.joinpath('out')
    if not scantailor_out_dir_path.is_dir():
        print(f'{scantailor_out_dir_path} does not exist')
        raise FileNotFoundError(f'{scantailor_out_dir_path} does not exist')
        
    # get list of images in "out" directory and copy to new_dir_path
    tif_path_list = list(scantailor_out_dir_path.glob('*.tif'))
    number_of_tifs = len(tif_path_list)
    if number_of_tifs == 0:
        print(f'{number_of_tifs} images in {scantailor_out_dir_path}')
        raise ValueError(f'{number_of_tifs} images in {scantailor_out_dir_path}')
        
    # create new directory in to_dir_path
    new_dir_path = to_dir_path.joinpath(from_dir_path.name)
    new_dir_path.mkdir(exist_ok=True)
    
    for tif_path in tif_path_list:
        new_tif_path = new_dir_path.joinpath(tif_path.name)
        if not new_tif_path.is_file():  # because we don't need to copy it if we already have!
            if decompress:  # then we check for compression
                image = Image.open(tif_path)
                decompress_continuous_tone(image, new_tif_path)
            else:
                copy2(tif_path, new_tif_path)

                
def move_scantailored(from_dir_path, holding_dir_path, qc_dir_path, decompress=True, batch=False):
    
    # if batch then we'll move all directories, otherwise, just move the 1st
    dir_path_list = sorted([x for x in from_dir_path.iterdir() if x.is_dir()])
    if not batch:  # let's just process the first in the list
        dir_path_list = [dir_path_list[0]]
    num_of_dirs = len(dir_path_list)
    if not num_of_dirs == 1:
        dirs = 'dirs'
    else:
        dirs = 'dir'
    print(f'Moving {num_of_dirs} {dirs} from {from_dir_path.name} to {holding_dir_path} . . .\n{"-" * 36}')
    for index, dir_path in enumerate(dir_path_list):
        new_dir_path = holding_dir_path.joinpath(dir_path.name)
        dir_path.rename(new_dir_path)
    print(f'. . . {index + 1} {dirs} moved. Last dir_path in list: {dir_path_list[-1]}\n')
    copy_holding_to_qc(holding_dir_path, qc_dir_path)
                

def copy_holding_to_qc(holding_dir_path, qc_dir_path, decompress=True, batch=True):
    
    # if batch then we'll move all directories, otherwise, just move the 1st
    dir_path_list = sorted([x for x in holding_dir_path.iterdir() if x.is_dir()])
    if not batch:  # let's just process the first in the list
        dir_path_list = [dir_path_list[0]]
    num_of_dirs = len(dir_path_list)
    if not num_of_dirs == 1:
        dirs = 'dirs'
    else:
        dirs = 'dir'
    print(f'Copying/verifying {num_of_dirs} {dirs} from {holding_dir_path.name} to {qc_dir_path.name} . . .\n{"-" * 36}')
    for index, dir_path in enumerate(dir_path_list):
        copy_scantailor_images(dir_path, qc_dir_path, decompress=decompress)
    print(f'. . . {index + 1} {dirs} copied/verified. Last dir modified: {max(dir_path_list, key=getmtime)}')

In [4]:
st = scantailor

In [5]:
move_scantailored(st.step1_dir_path, st.step2_dir_path, st.step3_dir_path)

Moving 1 dir from 1.toScanTailor to /Users/dlisla/shared_with_docker/agrutesc/2.ScanTailored-holding . . .
------------------------------------
. . . 1 dir moved. Last dir_path in list: /Users/dlisla/shared_with_docker/agrutesc/1.toScanTailor/0012_004268_000083

Copying/verifying 90 dirs from 2.ScanTailored-holding to 3.toQC . . .
------------------------------------
. . . 90 dirs copied/verified. Last dir modified: /Users/dlisla/shared_with_docker/agrutesc/2.ScanTailored-holding/0012_004268_000083
